In [5]:
import sys
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [6]:
project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_path not in sys.path:
    sys.path.append(project_path)

In [7]:
from src.modeling import train_model

## Modelo de regressão linear treinado, avaliado e salvo

In [16]:
# Definir caminhos dos arquivos
data_path = "../data/processed/dataset_imoveis_sp_clean.csv"
output_dir = "../outputs/models"
os.makedirs(output_dir, exist_ok=True)

# Carregar dataset
df = pd.read_csv(data_path)
print(f"Dataset carregado com {len(df)} registros.")

# Variáveis preditoras e alvo
features = ['area', 'bedrooms', 'garage', 'type_Apartamento', 'type_Casa', 'type_Casa em condomínio', 'type_Studio e kitnet']
alvo = 'rent'

# Dicionário para armazenar métricas
metrics_dict = {}

districts = df['district'].unique()
print(f"Encontrados {len(districts)} bairros únicos.")

for district in districts:
    print(f"Processando bairro: {district}")
    df_district = df[df['district'] == district].drop_duplicates()
    
    df_train = df_district[df_district['district'] != 'Outros']
    df_test = df_district.copy()
    
    if len(df_train) < 10:
        print(f"Poucos dados para o bairro {district}, pulando...")
        continue
    
    X_train, y_train = df_train[features], df_train[alvo]
    X_test, y_test = df_test[features], df_test[alvo]
    
    # Treinar o modelo
    model = train_model(X_train, y_train)
    print(f"Modelo treinado para {district}.")
    
    # Avaliar o modelo
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    metrics_dict[district] = {'R²': r2, 'RMSE': rmse, 'MAE': mae}
    print(f"Métricas para {district}: R²={r2:.4f}, RMSE={rmse:.2f}, MAE={mae:.2f}")
    
    # Salvar o modelo
    model_filename = os.path.join(output_dir, f'model_{district}.joblib')
    joblib.dump(model, model_filename)
    print(f"Modelo salvo em {model_filename}")

# Salvar métricas
metrics_df = pd.DataFrame.from_dict(metrics_dict, orient='index')
metrics_df.to_csv(os.path.join(output_dir, 'model_metrics.csv'))
print("Métricas salvas em outputs/model_metrics.csv")

# Calcular médias das métricas
if metrics_dict:
    avg_r2 = np.mean([m['R²'] for m in metrics_dict.values()])
    avg_rmse = np.mean([m['RMSE'] for m in metrics_dict.values()])
    avg_mae = np.mean([m['MAE'] for m in metrics_dict.values()])

    # Salvar médias no arquivo metrics.txt
    metrics_summary = f"Métricas Médias dos Modelos Treinados:\nR² médio: {avg_r2:.4f}\nRMSE médio: {avg_rmse:.2f}\nMAE médio: {avg_mae:.2f}\n"
    with open("../outputs/metrics.txt", "w") as f:
        f.write(metrics_summary)
    
    print("Métricas médias salvas em outputs/metrics.txt")


Dataset carregado com 11657 registros.
Encontrados 140 bairros únicos.
Processando bairro: Belenzinho
Modelo treinado para Belenzinho.
Métricas para Belenzinho: R²=0.7933, RMSE=947.22, MAE=710.87
Modelo salvo em ../outputs/models/model_Belenzinho.joblib
Processando bairro: Outros
Poucos dados para o bairro Outros, pulando...
Processando bairro: Pinheiros
Modelo treinado para Pinheiros.
Métricas para Pinheiros: R²=0.2734, RMSE=1927.71, MAE=1355.98
Modelo salvo em ../outputs/models/model_Pinheiros.joblib
Processando bairro: Bela Vista
Modelo treinado para Bela Vista.
Métricas para Bela Vista: R²=0.3887, RMSE=1334.23, MAE=900.54
Modelo salvo em ../outputs/models/model_Bela Vista.joblib
Processando bairro: Bras
Modelo treinado para Bras.
Métricas para Bras: R²=0.3340, RMSE=646.46, MAE=480.33
Modelo salvo em ../outputs/models/model_Bras.joblib
Processando bairro: Brooklin Paulista
Modelo treinado para Brooklin Paulista.
Métricas para Brooklin Paulista: R²=0.7322, RMSE=1938.29, MAE=1244.13
M